# Linear Regression Example

Let's walk through the steps of the official documentation example. Doing this will help your ability to read from the documentation, understand it, and then apply it to your own problems (the upcoming Consulting Project).

In [0]:
from pyspark.sql import SparkSession

In [0]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [0]:
from pyspark.ml.regression import LinearRegression

In [0]:
# Load training data
training = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

Interesting! We haven't seen libsvm formats before. In fact the aren't very popular when working with datasets in Python, but the Spark Documentation makes use of them a lot because of their formatting. Let's see what the training data looks like:

In [0]:
training.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
 -9.490009878824548|(10,[0,1,2,3,4,5,...|
 0.2577820163584905|(10,[0,1,2,3,4,5,...|
 -4.438869807456516|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
 -7.966593841555266|(10,[0,1,2,3,4,5,...|
 -7.896274316726144|(10,[0,1,2,3,4,5,...|
 -8.464803554195287|(10,[0,1,2,3,4,5,...|
 2.1214592666251364|(10,[0,1,2,3,4,5,...|
 1.0720117616524107|(10,[0,1,2,3,4,5,...|
-13.772441561702871|(10,[0,1,2,3,4,5,...|
 -5.082010756207233|(10,[0,1,2,3,4,5,...|
 7.887786536531237|(10,[0,1,2,3,4,5,...|
 14.323146365332388|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-0.8995693247765151|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
 5.601801561245534|(10,[0,1,2,3,4,5,...|
-3.2256352187273354|(10,[0,1,2,3,4,5,...|
 1.5299675726687754|(10,[0,1,2,3,4,5,...|
 -0.250102447941961|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

This is the format that Spark expects. Two columns with the names "label" and "features". 

The "label" column then needs to have the numerical label, either a regression numerical value, or a numerical value that matches to a classification grouping. Later on we will talk about unsupervised learning algorithms that by their nature do not use or require a label.

The feature column has inside of it a vector of all the features that belong to that row. Usually what we end up doing is combining the various feature columns we have into a single 'features' column using the data transformations we've learned about.

Let's continue working through this simple example!

In [0]:
# These are the default values for the featuresCol, labelCol, predictionCol
lr = LinearRegression(featuresCol='features', labelCol='label', predictionCol='prediction')

# You could also pass in additional parameters for regularization, do the reading 
# in ISLR to fully understand that, after that its just some simple parameter calls.
# Check the documentation with Shift+Tab for more info!

In [0]:
# Fit the model
lrModel = lr.fit(training)

In [0]:
# Print the coefficients and intercept for linear regression
print("Coefficients: {}".format(str(lrModel.coefficients))) # For each feature...
print('\n')
print("Intercept:{}".format(str(lrModel.intercept)))

Coefficients: [0.00733507102258,0.831375758434,-0.809530795468,2.44119168688,0.519171379529,1.15345919035,-0.298912411281,-0.51285141862,-0.619712827067,0.695615180432]


Intercept:0.14228558260358087

There is a summary attribute that contains even more info!

In [0]:
# Summarize the model over the training set and print out some metrics
trainingSummary = lrModel.summary

Lots of info, here are a few examples:

In [0]:
trainingSummary.residuals.show()
print("RMSE: {}".format(trainingSummary.rootMeanSquaredError))
print("r2: {}".format(trainingSummary.r2))

+-------------------+
 residuals|
+-------------------+
-11.011130022096554|
 0.9236590911176537|
-4.5957401897776675|
 -20.4201774575836|
-10.339160314788181|
-5.9552091439610555|
-10.726906349283922|
 2.122807193191233|
 4.077122222293812|
-17.316168071241652|
-4.5930443439590585|
 6.380476690746936|
 11.320566035059846|
-20.721971774534094|
 -2.736692773777402|
-16.668869342528467|
 8.242186378876315|
-1.3723486332690227|
-0.7060332131264666|
 -1.159113596999406|
+-------------------+
only showing top 20 rows

RMSE: 10.16309157133015
r2: 0.027839179518600154

## Train/Test Splits

But wait! We've commited a big mistake, we never separated our data set into a training and test set. Instead we trained on ALL of the data, something we generally want to avoid doing. Read ISLR and check out the theory lecture for more info on this, but remember we won't get a fair evaluation of our model by judging how well it does again on the same data it was trained on!

Luckily Spark DataFrames have an almost too convienent method of splitting the data! Let's see it:

In [0]:
all_data = spark.read.format("libsvm").load("sample_linear_regression_data.txt")

In [0]:
# Pass in the split between training/test as a list.
# No correct, but generally 70/30 or 60/40 splits are used. 
# Depending on how much data you have and how unbalanced it is.
train_data,test_data = all_data.randomSplit([0.7,0.3])

In [0]:
train_data.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
-28.571478869743427|(10,[0,1,2,3,4,5,...|
-28.046018037776633|(10,[0,1,2,3,4,5,...|
-26.736207182601724|(10,[0,1,2,3,4,5,...|
 -23.51088409032297|(10,[0,1,2,3,4,5,...|
-23.487440120936512|(10,[0,1,2,3,4,5,...|
-22.949825936196074|(10,[0,1,2,3,4,5,...|
-20.212077258958672|(10,[0,1,2,3,4,5,...|
-20.057482615789212|(10,[0,1,2,3,4,5,...|
-19.884560774273424|(10,[0,1,2,3,4,5,...|
-19.872991038068406|(10,[0,1,2,3,4,5,...|
-19.782762789614537|(10,[0,1,2,3,4,5,...|
-19.402336030214553|(10,[0,1,2,3,4,5,...|
 -19.16829262296376|(10,[0,1,2,3,4,5,...|
-18.845922472898582|(10,[0,1,2,3,4,5,...|
 -18.27521356600463|(10,[0,1,2,3,4,5,...|
-17.803626188664516|(10,[0,1,2,3,4,5,...|
-17.428674570939506|(10,[0,1,2,3,4,5,...|
 -17.32672073267595|(10,[0,1,2,3,4,5,...|
-17.026492264209548|(10,[0,1,2,3,4,5,...|
-16.692207021311106|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:
test_data.show()

+-------------------+--------------------+
 label| features|
+-------------------+--------------------+
-26.805483428483072|(10,[0,1,2,3,4,5,...|
-22.837460416919342|(10,[0,1,2,3,4,5,...|
-21.432387764165806|(10,[0,1,2,3,4,5,...|
 -19.66731861537172|(10,[0,1,2,3,4,5,...|
-17.494200356883344|(10,[0,1,2,3,4,5,...|
-17.065399625876015|(10,[0,1,2,3,4,5,...|
 -16.71909683360509|(10,[0,1,2,3,4,5,...|
 -16.08565904102149|(10,[0,1,2,3,4,5,...|
-15.732088272239245|(10,[0,1,2,3,4,5,...|
-13.420594775890757|(10,[0,1,2,3,4,5,...|
-12.977848725392104|(10,[0,1,2,3,4,5,...|
 -12.92222310337042|(10,[0,1,2,3,4,5,...|
-12.558575788856189|(10,[0,1,2,3,4,5,...|
-12.479280211451497|(10,[0,1,2,3,4,5,...|
 -12.46765638103286|(10,[0,1,2,3,4,5,...|
-12.198096564661412|(10,[0,1,2,3,4,5,...|
-11.904986902675114|(10,[0,1,2,3,4,5,...|
-11.857350365429426|(10,[0,1,2,3,4,5,...|
-11.615775265015627|(10,[0,1,2,3,4,5,...|
-11.328415936777782|(10,[0,1,2,3,4,5,...|
+-------------------+--------------------+
only showing top 20 rows

In [0]:
unlabeled_data = test_data.select('features')

In [0]:
unlabeled_data.show()

+--------------------+
 features|
+--------------------+
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
(10,[0,1,2,3,4,5,...|
+--------------------+
only showing top 20 rows

Now we only train on the train_data

In [0]:
correct_model = lr.fit(train_data)

Now we can directly get a .summary object using the evaluate method:

In [0]:
test_results = correct_model.evaluate(test_data)

In [0]:
test_results.residuals.show()
print("RMSE: {}".format(test_results.rootMeanSquaredError))

+-------------------+
 residuals|
+-------------------+
-27.947989982759562|
 -21.48974651218455|
-21.348863706083357|
 -19.11295339381348|
 -16.37056545252385|
-18.142472725017456|
 -16.58506142820683|
-13.240519904071881|
 -17.84000416941091|
-12.585063403174502|
 -15.18750826797708|
 -15.36121416276331|
-13.574499908567173|
-10.963252677902677|
-11.315618505456634|
-14.605252333044671|
-11.894926359717878|
-12.314571866864313|
-12.299976556997414|
 -11.33468035340431|
+-------------------+
only showing top 20 rows

RMSE: 10.1830320696341

Well that is nice, but realistically we will eventually want to test this model against unlabeled data, after all, that is the whole point of building the model in the first place. We can again do this with a convenient method call, in this case, transform(). Which was actually being called within the evaluate() method. Let's see it in action:

In [0]:
predictions = correct_model.transform(unlabeled_data)

In [0]:
predictions.show()

+--------------------+--------------------+
 features| prediction|
+--------------------+--------------------+
(10,[0,1,2,3,4,5,...| 1.1425065542764918|
(10,[0,1,2,3,4,5,...| -1.3477139047347926|
(10,[0,1,2,3,4,5,...|-0.08352405808245028|
(10,[0,1,2,3,4,5,...| -0.554365221558238|
(10,[0,1,2,3,4,5,...| -1.1236349043594933|
(10,[0,1,2,3,4,5,...| 1.07707309914144|
(10,[0,1,2,3,4,5,...|-0.13403540539825934|
(10,[0,1,2,3,4,5,...| -2.845139136949609|
(10,[0,1,2,3,4,5,...| 2.107915897171665|
(10,[0,1,2,3,4,5,...| -0.8355313727162558|
(10,[0,1,2,3,4,5,...| 2.209659542584976|
(10,[0,1,2,3,4,5,...| 2.4389910593928907|
(10,[0,1,2,3,4,5,...| 1.0159241197109834|
(10,[0,1,2,3,4,5,...| -1.5160275335488211|
(10,[0,1,2,3,4,5,...| -1.1520378755762275|
(10,[0,1,2,3,4,5,...| 2.4071557683832587|
(10,[0,1,2,3,4,5,...|-0.01006054295723...|
(10,[0,1,2,3,4,5,...| 0.45722150143488616|
(10,[0,1,2,3,4,5,...| 0.6842012919817876|
(10,[0,1,2,3,4,5,...|0.006264416626529094|
+--------------------+--------------------+
only showing top 20 rows

Okay, so this data is a bit meaningless, so let's explore this same process with some data that actually makes a little more intuitive sense!